In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px # choropleth maps
from IPython.display import Image
import os
import sys
import json
# plotly maps
# import plotly.figure_factory as ff
# from urllib.request import urlopen

# sys.path.append(os.path.abspath("C:/Users/Alex/Documents/My Data Files/COVID Research/analysis/mention_2_hashtag.py"))

# from analysis.functions.mention_2_hashtag import * 
# from analysis.functions.datamanip import *

# n grams
from sklearn.feature_extraction.text import CountVectorizer

raw = pd.read_csv('flair.joined.tweets.csv')

# changing date to more readable format
raw['created_at'] = pd.to_datetime(raw['created_at'])

# getting seperated date and time columns
raw['date'] = raw['created_at'].dt.date
raw['week'] = raw['created_at'].dt.week
raw['time'] = raw['created_at'].dt.time
raw['hour'] = raw['created_at'].dt.hour

min_wk = raw['week'].min()
max_wk = raw['week'].max()

min_date = raw['date'].min()
max_date = raw['date'].max()

# changing na to None
raw = raw.fillna('None')

raw = raw[raw['sentiment'] != 'None']

raw['is_negative'] = [1 if 'NEGATIVE' in str(sent) else 0 for sent in raw['sentiment']]
raw['is_positive'] = [1 if 'POSITIVE' in str(sent) else 0 for sent in raw['sentiment']]


def convert_fips(unknown_fips_list, fin):
    
    def convert(fipsline):
        name, postal, fips= fipsline.strip().split('\t')

        return {fips: name}

    fips_dict = {f: n 
                for dic in [convert(line)for line in open(fin, 'r')]
                for f, n in dic.items()}

    return [fips_dict[f'{unknown:02d}'] for unknown in unknown_fips_list]

raw = raw[raw['statefips'] != 'None']

raw['state_name'] = convert_fips([int(float(fips)) for fips in raw['statefips']],
                     '../analysis/functions/state.fips.txt')

us_state_abbrev = json.load(open('state_abbreviations.json'))

raw['state_abv'] = [us_state_abbrev[s] for s in raw['state_name']]

raw['fips'] = [f'{int(float(state)):02d}{int(float(county)):03d}' for i, state, county in raw[['statefips', 'countyfips']].itertuples()]
raw['count'] = 1
# raw.head()

all_splits = pd.to_datetime(['2020-03-05', '2020-03-26', '2020-05-01'])
raw['stage'] = ['s:1' if date < all_splits[0] else 's:2' if date < all_splits[1] else 's:3' if date < all_splits[2] else 's:4' for date in raw['date']]



#########################
# thresholds
thresholds = [0.8, 0.9, 0.95]
for thresh in thresholds:
    raw[f't:{thresh}'] = [1 if float(conf) > thresh else 0 for conf in raw['sent_confidence']]

col = 't:0.8'
raw['category'] = ['NEUT' if t == 0 else 'POS' if p == 1 else 'NEG' for i, n, p, t in raw[['is_negative', 'is_positive', col]].itertuples()]
raw['is_neutral'] = [1 if cat == 'NEUT' else 0 for cat in raw['category']]
raw['is_positive'] = [1 if cat == 'POS' else 0 for cat in raw['category']]
raw['is_negative'] = [1 if cat == 'NEG' else 0 for cat in raw['category']]

raw['is_neutral2'] = raw['is_neutral']
raw['is_positive2'] = raw['is_positive']
raw['is_negative2'] = raw['is_negative']

raw['is_neutral3'] = raw['is_neutral']
raw['is_positive3'] = raw['is_positive']
raw['is_negative3'] = raw['is_negative']

##################
# cdc

keywords = ['social distancing', 'social distance', 'physical distance', '6 feet', 'stay at home', 'school isolation', 'isolation', 'stay home', 'avoid touching', 'mask', 'covering', 'face shield', 'wear a mask', 'surgical mask', 'N95 respirator', 'wearing gloves', 'face shields', 'facial covering', 'skin protection', 'eye protection', 'ppe', 'wash hands', 'hand sanitizer', 'disinfect', 'clean', 'detergent', 'handwashing', 'hand hygiene', 'prevention hygiene', 'sprays', 'concentrates', 'wipes', 'routine cleaning', 'bleach solution', 'test', 'business closure']

topics = []
indexes = {i:0 for i in range(len(raw.index))}

for i, topic in enumerate(keywords):
    topics.append([1 if topic in text else 0 for text in raw['ogtext']])

for i, kw in enumerate(topics):
    for data_ind, value in enumerate(topics[i]):
        indexes[data_ind] += value

raw['in_cdc'] = [indexes[i] if 0 <= indexes[i] <= 1 else 1 for i in indexes]

cdc = raw[raw['in_cdc'] == 1]

cdc_splits = pd.to_datetime(['2020-03-12', '2020-04-20', '2020-05-14'])
cdc['stage'] = ['s:1' if date < cdc_splits[0] else 's:2' if date < cdc_splits[1] else 's:3' if date < cdc_splits[2] else 's:4' for date in cdc['date']]

# print(f'there are a total of {len(cdc.index)} tweets after subsetting for cdc keywords')
# print(raw)
# print(cdc)
print('loaded...')

loaded...


In [5]:
raw.columns

Index(['id_str', 'created_at', 'follower_count', 'friends_count', 'statefips',
       'countyfips', 'countyname', 'is_urban', 'ogtext', 'cleantext',
       'mentions', 'hashtags', 'pos', 'ner', 'sentiment', 'sent_confidence',
       'date', 'week', 'time', 'hour', 'is_negative', 'is_positive',
       'state_name', 'state_abv', 'fips', 'count', 'stage', 't:0.8', 't:0.9',
       't:0.95', 'category', 'is_neutral', 'is_neutral2', 'is_positive2',
       'is_negative2', 'is_neutral3', 'is_positive3', 'is_negative3',
       'in_cdc'],
      dtype='object')

In [6]:
keywords = ['social distancing', 'social distance', 'physical distance', '6 feet', 'stay at home', 'school isolation', 'isolation', 'stay home', 'avoid touching', 'mask', 'covering', 'face shield', 'wear a mask', 'surgical mask', 'N95 respirator', 'wearing gloves', 'face shields', 'facial covering', 'skin protection', 'eye protection', 'ppe', 'wash hands', 'hand sanitizer', 'disinfect', 'clean', 'detergent', 'handwashing', 'hand hygiene', 'prevention hygiene', 'sprays', 'concentrates', 'wipes', 'routine cleaning', 'bleach solution', 'test', 'business closure']

topics = []
indexes = {i:0 for i in range(len(raw.index))}

for i, topic in enumerate(keywords):
    topics.append([1 if topic in text else 0 for text in raw['ogtext']])

for i, kw in enumerate(topics):
    for data_ind, value in enumerate(topics[i]):
        indexes[data_ind] += value

raw['in_cdc'] = [indexes[i] if 0 <= indexes[i] <= 1 else 1 for i in indexes]

cdc = raw[raw['in_cdc'] == 1]

print(f'there are a total of {len(cdc.index)} tweets after subsetting for cdc keywords')

there are a total of 53272 tweets after subsetting for cdc keywords


# getting counties by rural/urban

In [7]:
df = raw.groupby(['fips']).agg({'count':'count', 'is_urban':'sum', 'is_negative':'mean', 'is_positive':'mean', 'is_neutral':'mean'}).reset_index()
df['is_rural'] = df['count'] - df['is_urban']
df['p_urban'] = df['is_urban'] / df['count']

df = df[df['count'] > 15]

# sns.regplot(y='is_negative', x='p_urban', data=df, marker='+')

In [8]:
# df.corr()**2

In [9]:
# import scipy
# x= df['p_urban']
# y= df['is_negative']

# print(scipy.stats.pearsonr(x, y))

# t test between urban and rural counties

In [11]:
# print(len(df.index))

# df2 = df[df['p_urban'] < 0.8]
# df2 = df2[df2['p_urban'] > 0.2]
# df2.describe()

In [21]:
import scipy

cats = 'negative positive neutral'.split(' ')

for cat in cats:
    print(f'{cat}:')

    urban = raw[raw['is_urban']==1].groupby(['fips']).agg({'count':'count', f'is_{cat}':'mean'}).reset_index()
    rural = raw[raw['is_urban']==0].groupby(['fips']).agg({'count':'count', f'is_{cat}':'mean'}).reset_index()

    urban = urban[urban['count'] > 15]
    rural = rural[rural['count'] > 15]

    print(f'len urban= {len(urban.index)}, len rural= {len(rural.index)}')
    print(f'meanUrban= {urban[f"is_{cat}"].mean()}, stdUrban= {urban[f"is_{cat}"].std()}')
    print(f'meanRural= {rural[f"is_{cat}"].mean()}, stdRural= {rural[f"is_{cat}"].std()}')

    from scipy.stats import ttest_ind
    t, p = ttest_ind(urban[f'is_{cat}'], rural[f'is_{cat}'], equal_var=False)
    print('         t                p')
    print(t, p)
    print()

# sns.distplot(urban['is_negative'], hist = True, kde = True,
#                  kde_kws = {'linewidth': 3}, label="urban")

# sns.distplot(rural['is_negative'], hist = True, kde = True,
#                  kde_kws = {'linewidth': 3}, label="rural", axlabel="proportion of negative sentiment")

# plt.title("COVID-19 Dataset")

negative:
len urban= 830, len rural= 182
meanUrban= 0.5768273405328975, stdUrban= 0.08968712292612052
meanRural= 0.5833472394796978, stdRural= 0.1063719154051495
         t                p
-0.7691173158706105 0.44257817384399456

positive:
len urban= 830, len rural= 182
meanUrban= 0.29213852840404303, stdUrban= 0.08271359810864314
meanRural= 0.29182402353422937, stdRural= 0.09948418419024763
         t                p
0.039743138239857694 0.9683311113931015

neutral:
len urban= 830, len rural= 182
meanUrban= 0.13103413106305942, stdUrban= 0.05051981034437689
meanRural= 0.12482873698607284, stdRural= 0.058787628429605734
         t                p
1.3210763469296816 0.18771866425205697



In [20]:
cats = 'negative positive neutral'.split(' ')

for cat in cats:
    print(f'{cat}:')

    urban = cdc[cdc['is_urban']==1].groupby(['fips']).agg({'count':'count', f'is_{cat}':'mean'}).reset_index()
    rural = cdc[cdc['is_urban']==0].groupby(['fips']).agg({'count':'count', f'is_{cat}':'mean'}).reset_index()

    urban = urban[urban['count'] > 15]
    rural = rural[rural['count'] > 15]

    print(f'len urban= {len(urban.index)}, len rural= {len(rural.index)}')
    print(f'meanUrban= {urban[f"is_{cat}"].mean()}, stdUrban= {urban[f"is_{cat}"].std()}')
    print(f'meanRural= {rural[f"is_{cat}"].mean()}, stdRural= {rural[f"is_{cat}"].std()}')

    from scipy.stats import ttest_ind
    t, p = ttest_ind(urban[f'is_{cat}'], rural[f'is_{cat}'], equal_var=False)
    print('         t                p')
    print(t, p)
    print()

negative:
len urban= 355, len rural= 52
meanUrban= 0.6112191650318727, stdUrban= 0.08908389553737095
meanRural= 0.6543482085240492, stdRural= 0.07851853162870684
         t                p
-3.6332091678131966 0.0005226805793239376

positive:
len urban= 355, len rural= 52
meanUrban= 0.24547082218192756, stdUrban= 0.08224891919147614
meanRural= 0.21730468428056784, stdRural= 0.07157451983033206
         t                p
2.5976012601735405 0.011369514256739112

neutral:
len urban= 355, len rural= 52
meanUrban= 0.1433100127861997, stdUrban= 0.05654815836476976
meanRural= 0.12834710719538311, stdRural= 0.04113606841943789
         t                p
2.321311160150001 0.02274569131763659

